In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import io
import time
from scipy.stats import linregress
from yahooquery import Ticker

In [ ]:
# Try to import yahooquery for fast filtering
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 0. HELPER FUNCTIONS (THE FIX)
# ==========================================
def calculate_z_score(info, fin, bs):
    """
    Calculates Altman Z-Score using Balance Sheet (bs) and Financials (fin).
    """
    try:
        # Get most recent data (column 0 is usually most recent in yfinance)
        total_assets = bs.loc['Total Assets'].iloc[0]
        total_liab = bs.loc['Total Liabilities Net Minority Interest'].iloc[0]
        current_assets = bs.loc['Current Assets'].iloc[0]
        current_liab = bs.loc['Current Liabilities'].iloc[0]
        retained_earnings = bs.loc['Retained Earnings'].iloc[0]
        
        # EBIT might be labeled differently
        if 'EBIT' in fin.index:
            ebit = fin.loc['EBIT'].iloc[0]
        elif 'Operating Income' in fin.index:
            ebit = fin.loc['Operating Income'].iloc[0]
        else:
            ebit = 0
            
        total_revenue = fin.loc['Total Revenue'].iloc[0]
        market_cap = info.get('marketCap', 0)

        # Handle missing Market Cap by using Price * Shares if needed, 
        # but info['marketCap'] is usually reliable.
        if market_cap is None: return 0

        # Altman Z-Score Components
        # A: Working Capital / Total Assets
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings / Total Assets
        B = retained_earnings / total_assets
        
        # C: EBIT / Total Assets
        C = ebit / total_assets
        
        # D: Market Value of Equity / Total Liabilities
        D = market_cap / total_liab
        
        # E: Sales / Total Assets
        E = total_revenue / total_assets

        # Standard Formula for Manufacturing (General use)
        z = (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
        return round(z, 2)
    except Exception:
        return 0.0

def get_margin_trend(fin):
    """
    Determines if Operating Margin is Improving, Declining, or Stable.
    Handles ZeroDivisionError if Revenue is 0.
    """
    try:
        # Need at least 2 years of data
        if fin.shape[1] < 2: return "Stable"
        
        # Current Year
        op_inc_curr = fin.loc['Operating Income'].iloc[0]
        rev_curr = fin.loc['Total Revenue'].iloc[0]
        
        # Previous Year
        op_inc_prev = fin.loc['Operating Income'].iloc[1]
        rev_prev = fin.loc['Total Revenue'].iloc[1]

        # FIX: Check for Zero Revenue to avoid crash/warnings
        if rev_curr == 0 or rev_prev == 0:
            return "Unknown"

        margin_curr = op_inc_curr / rev_curr
        margin_prev = op_inc_prev / rev_prev

        # Logic: >5% improvement counts as "Improving"
        if margin_curr > (margin_prev * 1.05): return "Improving"
        elif margin_curr < (margin_prev * 0.95): return "Declining"
        else: return "Stable"
    except Exception:
        return "Unknown"

def get_interest_coverage(fin):
    """
    Calculates EBIT / Interest Expense.
    """
    try:
        if 'EBIT' in fin.index:
            ebit = fin.loc['EBIT'].iloc[0]
        else:
            ebit = fin.loc['Operating Income'].iloc[0]
            
        # Interest Expense is often negative in dataframes, use abs()
        # Some companies have 'Interest Expense' or 'Interest Expense Net'
        if 'Interest Expense' in fin.index:
            int_exp = abs(fin.loc['Interest Expense'].iloc[0])
        elif 'Interest Expense Net' in fin.index:
            int_exp = abs(fin.loc['Interest Expense Net'].iloc[0])
        else:
            int_exp = 0
        
        if int_exp == 0: return 100.0 # No debt interest is safe
        
        return round(ebit / int_exp, 2)
    except Exception:
        return 0.0

def calculate_roic(fin, bs):
    """
    Calculates Return on Invested Capital (ROIC).
    """
    try:
        # NOPAT (Net Operating Profit After Tax)
        if 'EBIT' in fin.index:
            ebit = fin.loc['EBIT'].iloc[0]
        else:
            ebit = fin.loc['Operating Income'].iloc[0]
            
        tax_exp = fin.loc['Tax Provision'].iloc[0]
        pre_tax_income = fin.loc['Pretax Income'].iloc[0]
        
        # Effective Tax Rate
        if pre_tax_income != 0:
            tax_rate = tax_exp / pre_tax_income
        else:
            tax_rate = 0.21 # Default fallback
            
        nopat = ebit * (1 - tax_rate)
        
        # Invested Capital = Total Assets - Non-Interest Bearing Current Liabs
        # Simplified: Total Assets - Current Liabilities
        total_assets = bs.loc['Total Assets'].iloc[0]
        curr_liab = bs.loc['Current Liabilities'].iloc[0]
        
        invested_capital = total_assets - curr_liab
        
        if invested_capital == 0: return 0.0
        
        return round((nopat / invested_capital) * 100, 2)
    except Exception:
        return 0.0


In [3]:
# ==========================================
# 2. STEP 1: FETCH UNIVERSE (Robust Version)
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    
    # 1. USA (NASDAQ Traded List)
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        # Filter out Test issues and ETFs
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        # Filter for length < 5 to avoid warrants/rights generally
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")
        
    return list(set(tickers))

In [4]:
# ==========================================
# 3. STEP 2: SPLIT FILTER (With Debug Counts)
# ==========================================
def filter_universe_split(ticker_list):
    print(f"\n--- STEP 2: Applying Advanced Filters (YahooQuery) ---")
    
    # Configuration
    MIN_PRICE = 5.0
    MIN_CAP_US = 300_000_000
    MIN_VOL_US = 1_000_000
    MIN_CAP_CA = 100_000_000
    MIN_VOL_CA = 100_000
    
    MIN_CURRENT_RATIO = 1.0
    MIN_OP_MARGIN = 0.001
    MAX_PE = 100.0
    MAX_BETA = 3.0
    
    valid_candidates = []
    chunk_size = 500 
    
    # Counters for debugging
    cnt_us = 0
    cnt_ca = 0
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            # We fetch 'financialData' to get the recommendationKey
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("summaryDetail defaultKeyStatistics price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # Extract Data
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                beta = summ_mod.get('beta', 0) or 0
                pe_ratio = summ_mod.get('trailingPE', 0)
                
                fin_mod = data[symbol].get('financialData', {})
                curr_ratio = fin_mod.get('currentRatio', 0) or 0
                op_margin = fin_mod.get('operatingMargins', 0) or 0
                
                # --- NEW: Extract Rating & Handle "Strong Buy" formats ---
                rec_key = fin_mod.get('recommendationKey', 'none')
                if rec_key: 
                    rec_key = rec_key.lower().strip() # Clean string
                
                # Logic Filters
                if curr_price < MIN_PRICE: continue
                if curr_ratio < MIN_CURRENT_RATIO: continue
                if op_margin < MIN_OP_MARGIN: continue
                
                # --- NEW: Rating Filter ---
                # Yahoo often uses 'strong_buy' (with underscore). We allow both formats.
                if rec_key not in ['buy', 'strong_buy', 'strong buy']: continue

                # Region Logic
                is_canada = symbol.endswith('.TO')
                
                if is_canada:
                    if mkt_cap < MIN_CAP_CA: continue
                    if avg_vol < MIN_VOL_CA: continue
                else:
                    if mkt_cap < MIN_CAP_US: continue
                    if avg_vol < MIN_VOL_US: continue

                if MAX_BETA and beta > MAX_BETA: continue
                if MAX_PE and pe_ratio and pe_ratio > MAX_PE: continue

                # Add to list (Including Rating now)
                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Market_Cap': mkt_cap,
                    'Region': 'Canada' if is_canada else 'USA',
                    'Current_Ratio': curr_ratio,
                    'Op_Margin': op_margin,
                    'Rating': rec_key  # <--- Stored here
                })
                
                # Update Counters
                if is_canada: cnt_ca += 1
                else: cnt_us += 1
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    print(f"      - USA Survivors: {cnt_us}")
    print(f"      - Canada Survivors: {cnt_ca}")
    
    return df

In [5]:
# ==========================================
# 4. STEP 3: DEEP DIVE (Credit Model)
# ==========================================
def run_credit_model(candidates_df):
    if candidates_df.empty: return None, None, None
    
    print(f"\n--- STEP 3: Deep Analysis (Credit Model + Options Check) ---")
    
    fortress, moonshot, distress = [], [], []
    
    for index, row in candidates_df.iterrows():
        ticker = row['Ticker']
        
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(candidates_df)}...", end='\r')
            time.sleep(0.2)
            
        try:
            stock = yf.Ticker(ticker)
            
            # 1. Option Check
            try:
                if not stock.options: continue 
            except:
                continue 
            
            # 2. Data
            info = stock.info
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            if 'Financial' in info.get('sector', ''): continue
            
            # 3. Metrics
            z = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            # --- NEW: Include Rating in Final Output ---
            item = {
                'Ticker': ticker,
                'Rating': row.get('Rating', 'N/A'), # <--- Added Column
                'Price': row['Price'],
                'Region': row['Region'],
                'Z-Score': z,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Current_Ratio': row['Current_Ratio'],
                'Op_Margin': row['Op_Margin']
            }
            
            # 4. Buckets
            if (z > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress.append(item)
            elif (z < 1.8) or (int_cov < 1.5):
                distress.append(item)
            elif (z < 2.5) and (trend == "Improving"):
                moonshot.append(item)
                
        except:
            continue
            
    return pd.DataFrame(fortress), pd.DataFrame(moonshot), pd.DataFrame(distress)

In [ ]:
# --- MAIN EXECUTION ---
# 1. Get Universe
raw_tickers = get_raw_universe()

# 2. Split Filter (US vs Canada)
filtered_df = filter_universe_split(raw_tickers)

# 3. Run Analysis
if not filtered_df.empty:
    fortress_df, moonshot_df, distress_df = run_credit_model(filtered_df)
    
    if fortress_df is not None and not fortress_df.empty:
        print("\n\n--- FORTRESS STOCKS (Top Picks) ---")
        # Check if running in a notebook (display) or script (print)
        try:
            display(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
        except NameError:
            print(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
    
    if moonshot_df is not None and not moonshot_df.empty:
        print("\n--- MOONSHOT STOCKS ---")
        try:
            display(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))
        except NameError:
            print(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.

--- STEP 2: Applying Advanced Filters (YahooQuery) ---
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 429
      - USA Survivors: 429
      - Canada Survivors: 0

--- STEP 3: Deep Analysis (Credit Model + Options Check) ---
   Analyzing 425/429...

--- FORTRESS STOCKS (Top Picks) ---


,Ticker,Rating,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
6,NVDA,strong_buy,190.53,USA,91.42,Improving,341.19,78.13,4.468,0.63169
13,ISRG,buy,577.81,USA,57.85,Improving,100.00,12.08,4.728,0.30326
28,RGLD,buy,233.32,USA,46.35,Improving,44.71,10.28,3.516,0.50526
14,ADMA,strong_buy,19.28,USA,21.37,Improving,10.02,50.69,7.128,0.38005
31,DECK,buy,103.09,USA,12.84,Improving,354.51,34.60,3.067,0.22820
39,KLAC,buy,1279.60,USA,11.34,Improving,16.37,36.10,2.692,0.41661
41,PGNY,buy,25.80,USA,10.44,Stable,100.00,10.05,2.955,0.06870
43,SGHC,strong_buy,11.92,USA,9.69,Improving,32.04,18.80,1.750,0.23519
15,ZM,buy,88.04,USA,9.59,Improving,100.00,6.87,4.448,0.25240
9,RVLV,buy,30.42,USA,8.97,Improving,100.00,8.29,2.704,0.07436



--- MOONSHOT STOCKS ---


,Ticker,Rating,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
0,ITGR,buy,78.88,USA,2.49,Improving,3.82,5.75,3.709,0.14284
5,NRG,buy,160.88,USA,2.42,Improving,3.22,10.72,1.053,0.05527
6,AXTA,buy,32.42,USA,2.38,Improving,3.42,9.37,2.199,0.15761
2,HMY,buy,21.74,USA,2.25,Improving,15.79,17.47,1.720,0.29669
4,BKR,buy,45.25,USA,2.20,Improving,17.49,12.57,1.409,0.13524
8,RYN,buy,21.78,USA,2.17,Improving,11.19,13.13,3.443,0.27452
7,NOV,buy,15.62,USA,2.04,Improving,10.13,7.82,2.545,0.04917
3,WH,buy,76.65,USA,1.92,Improving,3.85,10.39,1.157,0.47382
1,AA,buy,54.25,USA,1.89,Improving,2.85,0.35,1.561,0.02771


In [9]:
import os
import time

# ==========================================
# 6. SAVE GENERAL RESULTS TO EXCEL
# ==========================================
def save_general_analysis_to_excel(fortress, moonshot, distress):
    """
    Saves the Fortress, Moonshot, and Distress lists to separate Excel sheets.
    Target: OneDrive folder with Date Stamp.
    """
    # Check if we have any data to save
    if (fortress is None or fortress.empty) and \
       (moonshot is None or moonshot.empty) and \
       (distress is None or distress.empty):
        print("No data to save.")
        return

    # --- 1. SETUP DATE & FILENAME ---
    today_date = time.strftime("%Y-%m-%d")
    file_nickname = f"General Market Research {today_date}.xlsx"

    # --- 2. SPECIFY DESTINATION ---
    # We use the explicit path you provided. 
    # Note: We use a raw string (r"...") to handle backslashes correctly on Windows.
    onedrive_folder = r"C:\Users\James\OneDrive - McMaster University\YFinance Stock Picks\Nonfinancial Stocks"
    
    # Combine folder and filename
    full_path = os.path.join(onedrive_folder, file_nickname)

    print(f"Saving results to: {full_path}...")

    try:
        # Check if directory exists; if not, create it to avoid errors
        if not os.path.exists(onedrive_folder):
            print(f"   Warning: Folder not found. Creating: {onedrive_folder}")
            os.makedirs(onedrive_folder, exist_ok=True)

        with pd.ExcelWriter(full_path, engine='openpyxl') as writer:
            
            # Sheet 1: Fortress (Sorted by Safety/Z-Score)
            if fortress is not None and not fortress.empty:
                fortress.sort_values(by='Z-Score', ascending=False).to_excel(
                    writer, sheet_name='Fortress (Safe)', index=False
                )
                print(f"   -> Saved {len(fortress)} Fortress stocks.")
            
            # Sheet 2: Moonshot (Sorted by Z-Score)
            if moonshot is not None and not moonshot.empty:
                moonshot.sort_values(by='Z-Score', ascending=False).to_excel(
                    writer, sheet_name='Moonshot (Growth)', index=False
                )
                print(f"   -> Saved {len(moonshot)} Moonshot stocks.")
            
            # Sheet 3: Distress (Sorted by Risk/Z-Score ascending)
            if distress is not None and not distress.empty:
                distress.sort_values(by='Z-Score', ascending=True).to_excel(
                    writer, sheet_name='Distress (Avoid)', index=False
                )
                print(f"   -> Saved {len(distress)} Distress stocks.")
                
        print(f"✅ Success! File saved.")

    except Exception as e:
        print(f"❌ Error saving file: {e}")
        print("Check if the file is open in Excel or if the path is correct.")

# --- EXECUTION ---
# This block runs immediately after your analysis finishes
if 'fortress_df' in locals() and 'moonshot_df' in locals():
    save_general_analysis_to_excel(fortress_df, moonshot_df, distress_df)
else:
    print("Please run the analysis step first to generate the dataframes.")

Saving results to: C:\Users\James\OneDrive - McMaster University\YFinance Stock Picks\Nonfinancial Stocks\General Market Research 2025-12-28.xlsx...
   -> Saved 46 Fortress stocks.
   -> Saved 9 Moonshot stocks.
   -> Saved 44 Distress stocks.
✅ Success! File saved.


In [ ]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")